In [64]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
from datetime import datetime, timedelta, date
import yaml
import requests

In [65]:
# Latitude and longitude
lat = 39.3999
long = -8.2245

In [66]:
# Start and end date
start_date = "1990-01-01"
end_date = "2024-12-14"

In [67]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [81]:
# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": lat,
	"longitude": long,
	"start_date": start_date,
	"end_date": end_date,
	"hourly": ["temperature_2m", "relative_humidity_2m"]
}
responses = openmeteo.weather_api(url, params=params)

In [82]:
# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")


Coordinates 39.40245819091797°N -8.204437255859375°E
Elevation 170.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [83]:
# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()

In [84]:
hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

In [85]:
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m

In [86]:
hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

                            date  temperature_2m  relative_humidity_2m
0      1990-01-01 00:00:00+00:00          9.4725             98.002426
1      1990-01-01 01:00:00+00:00          9.4725             98.332886
2      1990-01-01 02:00:00+00:00          9.5725             98.665382
3      1990-01-01 03:00:00+00:00          9.8225             98.668015
4      1990-01-01 04:00:00+00:00         10.0225             98.670097
...                          ...             ...                   ...
306403 2024-12-14 19:00:00+00:00             NaN                   NaN
306404 2024-12-14 20:00:00+00:00             NaN                   NaN
306405 2024-12-14 21:00:00+00:00             NaN                   NaN
306406 2024-12-14 22:00:00+00:00             NaN                   NaN
306407 2024-12-14 23:00:00+00:00             NaN                   NaN

[306408 rows x 3 columns]


In [87]:
def fetch_sunrise_sunset(lat, lon, date):
    """
    Fetch sunrise and sunset times for a given latitude, longitude, and date using Sunrise-Sunset API.
    """
    
    url = "https://api.sunrise-sunset.org/json"
    params = {
        'lat': lat,
        'lng': lon,
        'date': date,
        'formatted': 0  # Return times in ISO 8601 format (UTC)
    }
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        if data['status'] == 'OK':
            return {
                'date': date,
                'sunrise': data['results']['sunrise'],
                'sunset': data['results']['sunset']
            }
        else:
            print(f"Error: {data['status']}")
            return None
    else:
        print(f"HTTP Error: {response.status_code}")
        return None

In [88]:
def fetch_sunrise_sunset_range(lat, lon, start_date, end_date):
    """
    Fetch sunrise and sunset times for a range of dates.
    """
    # Convert start_date and end_date to datetime.date objects if they are strings
    if isinstance(start_date, str):
        start_date = datetime.strptime(start_date, "%Y-%m-%d").date()
    if isinstance(end_date, str):
        end_date = datetime.strptime(end_date, "%Y-%m-%d").date()

    # Ensure current_date starts as a datetime.date object
    current_date = start_date
    results = []

    while current_date <= end_date:
        print(f"Fetching data for {current_date}...")
        # Pass the date as a string in 'YYYY-MM-DD' format to the fetch function
        date_str = current_date.strftime("%Y-%m-%d")
        data = fetch_sunrise_sunset(lat, lon, date_str)
        if data:
            results.append(data)
        # Increment current_date by one day
        current_date += timedelta(days=1)

    return results

In [89]:
# Test
# get_sunset = fetch_sunrise_sunset(lat, long, "2024-12-10")

In [ ]:
# Load all historical sunrise/sunset data
historical_sunset_data = fetch_sunrise_sunset_range(lat, long, start_date, end_date)

Fetching data for 1990-01-01...
Fetching data for 1990-01-02...
Fetching data for 1990-01-03...
Fetching data for 1990-01-04...
Fetching data for 1990-01-05...
Fetching data for 1990-01-06...
Fetching data for 1990-01-07...
Fetching data for 1990-01-08...
Fetching data for 1990-01-09...
Fetching data for 1990-01-10...
Fetching data for 1990-01-11...
Fetching data for 1990-01-12...
Fetching data for 1990-01-13...
Fetching data for 1990-01-14...
Fetching data for 1990-01-15...
Fetching data for 1990-01-16...
Fetching data for 1990-01-17...
Fetching data for 1990-01-18...
Fetching data for 1990-01-19...
Fetching data for 1990-01-20...
Fetching data for 1990-01-21...
Fetching data for 1990-01-22...
Fetching data for 1990-01-23...
Fetching data for 1990-01-24...
Fetching data for 1990-01-25...
Fetching data for 1990-01-26...
Fetching data for 1990-01-27...
Fetching data for 1990-01-28...
Fetching data for 1990-01-29...
Fetching data for 1990-01-30...
Fetching data for 1990-01-31...
Fetching

In [ ]:
# Convert to table


In [ ]:
# Write to .csv